In [ ]:
# ===== Импорты и инициализация =====

#!pip install scikit-learn

from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.rt_detr import RT_DETR
from grid.model.perception.depth.metric3d import Metric3D

import airgen
import numpy as np
import rerun as rr
import cv2
import time
from math import cos, sin, pi
from sklearn.cluster import DBSCAN

# Инициализация моделей
nav_inspect = ObjectInspect()
nav_visualservoing = VisualServoing()
detection_rt_detr = RT_DETR(use_local=True)  # Инициализация модели детекции

In [ ]:
# ===== Подключение к дрону и взлёт =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
# ===== Взлет дрона =====

client.takeoffAsync().join()

# ===== Посадка дрона =====

#client.landAsync().join()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [ ]:
# ===== Проверка установленных камер ALTA X =====

scene_client = airgen.VehicleClient()
scene_objects = scene_client.simListSceneObjects()
cameras = [obj for obj in scene_objects if 'camera' in obj.lower()]

print("Available cameras:")
for camera in cameras:
    print(camera)

Available cameras:
PlayerCameraManager_2147482464
CameraManagerActor
ExternalCamera
BP_PIPCamera_C_2147482424
BP_PIPCamera_C_2147482412
BP_PIPCamera_C_2147482400
BP_PIPCamera_C_2147482388
BP_PIPCamera_C_2147482376


In [ ]:
# ===== Получение данных LiDAR + визуализация =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
rr.log('lidar/points', rr.Points3D(points))

In [ ]:
# ===== Получение и логирование изображения с камеры =====

image_data = client.getImages("front_center", [airgen.ImageType.Scene])[0]
rgb_image, _ = image_data  # Получаем изображение и позу
rr.log("drone/camera", rr.Image(rgb_image))

In [ ]:
# ===== Получение и логирование GPS =====

gps_data = client.getGpsData()
geo = gps_data.gnss.geo_point
gps_position = np.array([[geo.latitude, geo.longitude, geo.altitude]], dtype=np.float32)
rr.log("drone/gps", rr.Points3D(gps_position))

In [ ]:
# ===== Инициализация =====
depth_model = Metric3D()
depth_map = depth_model.run(rgb_image)

result = detection_rt_detr.run(input=rgb_image.copy(), confidence_threshold=0.3)

if isinstance(result, dict) and 'boxes' in result:
    filtered_boxes = []
    filtered_labels = []
    filtered_distances = []

    ignore_labels = ["grass", "floor", "ground", "road", "terrain"]
    min_distance = 2
    max_distance = 20

    # Центр изображения — точка откуда "исходит БПЛА"
    img_center = (rgb_image.shape[1] // 2, rgb_image.shape[0] // 2)

    image_with_lines = rgb_image.copy()

    for box, label in zip(result["boxes"], result["labels"]):
        x, y, w, h = map(int, box)
        cx = x + w // 2
        cy = y + h // 2

        if 0 <= cy < depth_map.shape[0] and 0 <= cx < depth_map.shape[1]:
            distance = depth_map[cy, cx]

            if (min_distance <= distance <= max_distance) and (label.lower() not in ignore_labels):
                filtered_boxes.append([x, y, w, h])
                filtered_labels.append(label)
                filtered_distances.append(distance)

                # ===== Отрисовка линии до объекта =====
                line_color = (0, 255, 0) if distance > 10 else (0, 255, 255) if distance > 5 else (0, 0, 255)
                cv2.line(image_with_lines, img_center, (cx, cy), line_color, 2)

                # Подпись расстояния у центра объекта
                cv2.putText(image_with_lines, f"{distance:.1f}m", (cx + 5, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, line_color, 2)

    # ===== Отрисовка боксов =====
    for i, box in enumerate(filtered_boxes):
        x, y, w, h = box
        cv2.rectangle(image_with_lines, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(image_with_lines, filtered_labels[i], (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        rr.log(
            f"drone/object_{i}",
            rr.Boxes2D(np.array([[x, y, w, h]]), labels=[f"{filtered_labels[i]} - {filtered_distances[i]:.1f}m"])
        )

    # ===== Лог финального изображения =====
    rr.log("drone/camera", rr.Image(image_with_lines))

else:
    print("❗ Объекты не обнаружены.")

❗ Объекты не обнаружены.


In [ ]:
# ===== Получение и подготовка данных LiDAR =====
lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# ===== Фильтрация: исключаем "землю" (высота < 0.2 м) =====
above_ground = points[points[:, 2] > 0.2]

# ===== Кластеризация (DBSCAN) =====
clustering = DBSCAN(eps=1.0, min_samples=10).fit(above_ground[:, :2])  # кластеризация по X, Y
labels = clustering.labels_

# ===== Обработка кластеров =====
unique_labels = set(labels)
object_centers = []
object_distances = []

for label in unique_labels:
    if label == -1:
        continue  # -1 означает шум

    cluster_points = above_ground[labels == label]
    center = cluster_points.mean(axis=0)
    distance = np.linalg.norm(center[:2])
    
    object_centers.append(center)
    object_distances.append(distance)

# ===== Логгирование =====
rr.log("lidar/points", rr.Points3D(points))
rr.log("lidar/objects", rr.Points3D(np.array(object_centers)))

print(f"🔎 Обнаружено объектов по LiDAR: {len(object_centers)}")
for i, dist in enumerate(object_distances):
    print(f" - Объект {i+1}: ~{dist:.1f} м")


🔎 Обнаружено объектов по LiDAR: 9
 - Объект 1: ~11.9 м
 - Объект 2: ~12.0 м
 - Объект 3: ~46.0 м
 - Объект 4: ~12.0 м
 - Объект 5: ~11.9 м
 - Объект 6: ~8.0 м
 - Объект 7: ~10.2 м
 - Объект 8: ~10.3 м
 - Объект 9: ~0.1 м


In [ ]:
# ===== Построение и визуализация кругового маршрута =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))

# Стартовая точка
start = client.getMultirotorState().kinematics_estimated.position
takeoff_alt = -15  # минус = вверх в NED
center = start
radius = 10
steps = 12  # количество точек круга

# Построение круга
trajectory = []
for i in range(steps):
    angle = 2 * pi * (i / steps)
    x = center.x_val + radius * cos(angle)
    y = center.y_val + radius * sin(angle)
    z = takeoff_alt
    trajectory.append(airgen.Vector3r(x, y, z))

# Возврат к центру
trajectory.append(airgen.Vector3r(center.x_val, center.y_val, takeoff_alt))

# Визуализация маршрута
rr.log("telemetry/path", rr.LineStrips3D([[(p.x_val, p.y_val, p.z_val) for p in trajectory]]))

In [ ]:
# ===== Плавное движение по маршруту и посадка =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))
velocity = 3.0  # Плавная скорость

client.moveOnPathAsync(
    trajectory,
    velocity,
    300,
    airgen.DrivetrainType.ForwardOnly,
    airgen.YawMode(False, 0),
    -1, 0
).join()

# Плавная посадка
landing_point = airgen.Vector3r(center.x_val, center.y_val, -2)
client.moveToPositionAsync(landing_point.x_val, landing_point.y_val, landing_point.z_val, 1).join()